# GPU Accelerated SVCs for Graph Classification using the GNTK

## Notebook Configuration
---
Install `Cuda 9.0` for linux_x86_64 in order to enable full compatibility with the `thundersvm` package.

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

--2021-01-10 09:55:07--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.199.0.24
Connecting to developer.nvidia.com (developer.nvidia.com)|152.199.0.24|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?0Z5Hvz7MGdbgZKTl7zMcuOE76jrWYr0WWPww_dBx-RNchjuttcWaVWuV0X6UbvxGJgbX30VNqIei2AcHKPyViapGAKi1VLOhtO-DTlLhuw8zxDlK6nZCPu4gnuO9BceRL2XLink747wBiSP3K54jscnqO6ZaCJcTBN46oCxo93Ccm5XMyfKRWNUUg_m8JmwbLJlDe44KpZo1neDtJshQ [following]
--2021-01-10 09:55:08--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?0Z5Hvz7MGdbgZKTl7zMcuOE76jrWYr0WWPww_dBx-RNchjuttcWaVWuV0X6UbvxGJgbX30VNqIei2AcHKPyViapGAKi1VLOhtO-DTlLhuw8zxDlK6nZC

In [2]:
!ls  ## Check if required cuda 9.0 amd64-deb file is downloaded

cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb  IMDBBINARY.zip  sample_data


In [3]:
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb

Selecting previously unselected package cuda-repo-ubuntu1704-9-0-local.
(Reading database ... 145483 files and directories currently installed.)
Preparing to unpack cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb ...
Unpacking cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...
Setting up cuda-repo-ubuntu1704-9-0-local (9.0.176-1) ...


In [4]:
!ls /var/cuda-repo-9-0-local | grep .pub

7fa2af80.pub


In [5]:
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub

OK


In [6]:
!apt-get update

Get:1 file:/var/cuda-repo-9-0-local  InRelease
Ign:1 file:/var/cuda-repo-9-0-local  InRelease
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:2 file:/var/cuda-repo-9-0-local  Release [574 B]
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]
Get:3 file:/var/cuda-repo-9-0-local  Release.gpg [819 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:10 file:/var/cuda-repo-9-0-local  Packages [15.8 kB]
Get:11 http://security.ubuntu.com

In [7]:
!sudo apt-get install cuda-9.0

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'cuda-9-0' for regex 'cuda-9.0'
Note, selecting 'libcuda-9.0-1' for regex 'cuda-9.0'
The following additional packages will be installed:
  cuda-command-line-tools-9-0 cuda-core-9-0 cuda-cublas-9-0
  cuda-cublas-dev-9-0 cuda-cudart-9-0 cuda-cudart-dev-9-0 cuda-cufft-9-0
  cuda-cufft-dev-9-0 cuda-curand-9-0 cuda-curand-dev-9-0 cuda-cusolver-9-0
  cuda-cusolver-dev-9-0 cuda-cusparse-9-0 cuda-cusparse-dev-9-0
  cuda-demo-suite-9-0 cuda-documentation-9-0 cuda-driver-dev-9-0
  cuda-libraries-9-0 cuda-libraries-dev-9-0 cuda-license-9-0
  cuda-misc-headers-9-0 cuda-npp-9-0 cuda-npp-dev-9-0 cuda-nvgraph-9-0
  cuda-nvgraph-dev-9-0 cuda-nvml-dev-9-0 cuda-nvrtc-9-0 cuda-nvrtc-dev-9-0
  cuda-runtime-9-0 cuda-samples-9-0 cuda-toolkit-9-0 cuda-visual-tools-9-0
The following NEW packages will be installed:
  cuda-9-0 cuda-command-line-tools-9-0 cuda-core-9-0 cuda-cublas-9-0
  cuda-cublas-de

In [8]:
# Check if Cuda 9.0 is successfully installed
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


Now we can procede to install `thundersvm` (we will use the GPU version in order to speedup training).

In [9]:
!pip install thundersvm

     |████████████████████████████████| 512kB 7.7MB/s 


## Fitting the models
---

In [11]:
!unzip IMDBBINARY.zip

Archive:  IMDBBINARY.zip
   creating: IMDBBINARY/
  inflating: IMDBBINARY/blocks1_layers1_jkn_uniform_gram.npy  
  inflating: IMDBBINARY/blocks2_layers2_sum_degree_gram.npy  
  inflating: IMDBBINARY/blocks3_layers3_jkn_uniform_labels.npy  
  inflating: IMDBBINARY/blocks2_layers2_sum_degree_labels.npy  
  inflating: IMDBBINARY/blocks2_layers1_jkn_uniform_labels.npy  
  inflating: IMDBBINARY/blocks2_layers3_jkn_degree_labels.npy  
  inflating: IMDBBINARY/blocks2_layers3_sum_degree_gram.npy  
  inflating: IMDBBINARY/blocks1_layers1_sum_uniform_labels.npy  
  inflating: IMDBBINARY/blocks2_layers1_sum_degree_gram.npy  
  inflating: IMDBBINARY/blocks1_layers2_sum_uniform_gram.npy  
  inflating: IMDBBINARY/blocks1_layers2_sum_uniform_labels.npy  
  inflating: IMDBBINARY/blocks2_layers1_jkn_degree_labels.npy  
  inflating: IMDBBINARY/.DS_Store    
   creating: __MACOSX/
   creating: __MACOSX/IMDBBINARY/
  inflating: __MACOSX/IMDBBINARY/._.DS_Store  
  inflating: IMDBBINARY/blocks3_layers2_jkn_

Package imports.

In [1]:
import os
import time
import thundersvm
import numpy as np
import pandas as pd

from google.colab import files
from tqdm.notebook import tqdm
#from multiprocessing import Process

Define hyper-parameters for the notebook and define the folds for k-fold cross validation.

In [2]:
# Specify the dataset for the experiments
DATASET = 'IMDBBINARY'
# Specify output directory
RESULTS_PATH = f'{DATASET}/results'
if not os.path.exists(RESULTS_PATH):
    os.mkdir(RESULTS_PATH)
# Specify the model to be evaluated. If None is given, take all the models
# inside the specified directory
MODEL_TYPE = []
if not len(MODEL_TYPE):
    MODEL_TYPE = [m[:-9] for m in os.listdir(DATASET) if m.endswith('_gram.npy')]

# Parameters for k-fold cross validation
K = 10
C_list = np.logspace(-2, 4, 64)
# Define folds for cross validation
n, _ = np.load(f'{DATASET}/{MODEL_TYPE[0]}_gram.npy').shape
permutation = np.random.permutation(n)
folds = np.array_split(permutation, K)

Fit the model and perform grid search in the `C_list` defined space.

In [3]:
# Fit a k-fold cross validated SVC for every GNTK version
for model_name in MODEL_TYPE:
    print()
    print('-' * 30)
    print(f'Performing k-fold cross validation with {model_name}...')
    # Read the gram matrix
    gram = np.load(f'{DATASET}/{model_name}_gram.npy')
    gram = gram / (gram.min() + 1e-06)
    labels = np.load(f'{DATASET}/{model_name}_labels.npy')
    # Perform symmetric normalization
    gram_diag = np.sqrt(np.diag(gram))
    gram = gram / gram_diag[:, None] / gram_diag[None, :]
    # k-fold cross validation
    results = []
    # Perform grid search
    for C in tqdm(C_list):
        C_results = []
        C_train_time = []
        C_score_time = []
        # Fit for each of the folds
        for i in range(K):
            # Determine indices for train/test
            test_indices = np.sort(folds[i])
            train_indices = np.array([
                j for j in np.arange(n, dtype=int) 
                if j not in test_indices
            ])
            # The Gram matrix for training will be the matrix containing the kernel
            # values between trainin observations
            gram_train = \
                np.take(np.take(gram, train_indices, axis=0), train_indices, axis=1)
            labels_train = labels[train_indices]
            # The Gram matrix for testing will be filled with the kernel values
            # computed between the test and train data
            gram_test = \
                np.take(np.take(gram, test_indices, axis=0), train_indices, axis=1)
            labels_test = labels[test_indices]
            # Init the classifier
            clf = thundersvm.SVC(kernel='precomputed', C=C, max_iter=1000)
            # Fit the model to training data
            start = time.time()
            clf.fit(gram_train, labels_train)
            end1 = time.time()
            # Predict on testing data
            C_results.append(clf.score(gram_test, labels_test))
            end2 = time.time()
            C_train_time.append(end1 - start)
            C_score_time.append(end2 - end1)
        # Save results
        results.append({
            'C': C,
            'accuracy_mean': np.mean(C_results),
            'accuracy_std': np.std(C_results),
            'train_time_mean': np.mean(C_train_time),
            #'train_time_std': np.std(C_train_time),
            'score_time_mean': np.mean(C_score_time),
            #'score_time_std': np.std(C_score_time)
        })
    results = pd.DataFrame(results)
    # Save the results as a csv file
    results_filename = f'{RESULTS_PATH}/{model_name}_normalized'
    results.to_csv(f'{results_filename}_results.csv', index=False)
    # Verbose the best achieved results
    max_mean_score = results['accuracy_mean'].max()
    print('Best result:')
    display(results[results['accuracy_mean'] == max_mean_score])


------------------------------
Performing k-fold cross validation with blocks3_layers1_sum_uniform...



Best result:


,C,accuracy_mean,accuracy_std,train_time_mean,score_time_mean
63,10000.0,0.749,0.024678,2.351359,0.017064



------------------------------
Performing k-fold cross validation with blocks3_layers2_jkn_degree...



Best result:


...


Best result:


,C,accuracy_mean,accuracy_std,train_time_mean,score_time_mean
46,240.409918,0.735,0.036125,0.138677,0.016514



------------------------------
Performing k-fold cross validation with blocks3_layers2_jkn_uniform...



Best result:


,C,accuracy_mean,accuracy_std,train_time_mean,score_time_mean
52,896.150502,0.75,0.02569,0.11139,0.016242


In [6]:
# Zip the results and download for further investigations
!zip -r results.zip IMDBBINARY/results

  adding: IMDBBINARY/results/ (stored 0%)
  adding: IMDBBINARY/results/blocks1_layers3_sum_uniform_normalized_results.csv (deflated 58%)
  adding: IMDBBINARY/results/blocks3_layers2_jkn_uniform_normalized_results.csv (deflated 59%)
  adding: IMDBBINARY/results/blocks2_layers1_jkn_degree_normalized_results.csv (deflated 58%)
  adding: IMDBBINARY/results/blocks2_layers2_jkn_degree_normalized_results.csv (deflated 58%)
  adding: IMDBBINARY/results/blocks3_layers1_sum_uniform_normalized_results.csv (deflated 57%)
  adding: IMDBBINARY/results/blocks1_layers2_sum_degree_normalized_results.csv (deflated 57%)
  adding: IMDBBINARY/results/blocks1_layers1_jkn_degree_normalized_results.csv (deflated 58%)
  adding: IMDBBINARY/results/.ipynb_checkpoints/ (stored 0%)
  adding: IMDBBINARY/results/blocks2_layers3_jkn_uniform_normalized_results.csv (deflated 58%)
  adding: IMDBBINARY/results/blocks2_layers2_sum_degree_normalized_results.csv (deflated 59%)
  adding: IMDBBINARY/results/blocks2_layers2_jk